<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Hands-on Lab: Interactive Visual Analytics with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [2]:
import folium
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


In [4]:
## Task 1: Mark all launch sites on a map
# Create a map centered on a general location of the launch sites
site_map = folium.Map(location=[28.573255, -80.646895], zoom_start=5)

# Dictionary containing launch site names and their coordinates
launch_sites = {
    "CCAFS SLC-40": [28.563197, -80.576820],
    "VAFB SLC-4E": [34.632834, -120.610746],
    "KSC LC-39A": [28.57205, -80.64800],
    "CCAFS LC-40": [28.561857, -80.577366]
}

# Add markers to the map for each launch site
for site, coord in launch_sites.items():
    folium.Marker(coord, popup=site, icon=folium.Icon(color="blue", icon="info-sign")).add_to(site_map)

# Display the map
site_map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


In [5]:
print(launch_sites)

{'CCAFS SLC-40': [28.563197, -80.57682], 'VAFB SLC-4E': [34.632834, -120.610746], 'KSC LC-39A': [28.57205, -80.648], 'CCAFS LC-40': [28.561857, -80.577366]}


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [6]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

In [7]:
spacex_df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


Now, you can take a look at what are the coordinates for each site.


In [8]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [9]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [10]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [13]:
# Add a circle for each launch site and a marker with a label
for index, row in launch_sites_df.iterrows():
    # Extract coordinates
    launch_site_name = row['Launch Site']
    lat = row['Lat']
    lon = row['Long']
    
    # Add a larger circle at the launch site
    folium.Circle(
        location=[lat, lon],
        radius=1000,  # Increased radius for better visibility
        color='orange',  # Change to orange for better visualization
        fill=True,
        fill_color='orange'
    ).add_to(site_map)
    
    # Add a marker with a popup label
    folium.Marker(
        location=[lat, lon],
        popup=launch_site_name,
        tooltip=launch_site_name,  # Show name on hover
        icon=folium.Icon(color="blue", icon="info-sign")
    ).add_to(site_map)

# Display the updated map
site_map


The generated map with marked launch sites should look similar to the following:


In [12]:
print(launch_sites_df)

    Launch Site        Lat        Long
0   CCAFS LC-40  28.562302  -80.577356
1  CCAFS SLC-40  28.563197  -80.576820
2    KSC LC-39A  28.573255  -80.646895
3   VAFB SLC-4E  34.632834 -120.610745


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?

No, the launch sites are not all near the Equator (Latitude 0°).
However, the sites in Florida (CCAFS LC-40, KSC LC-39A) are relatively closer to the Equator than Vandenberg (VAFB SLC-4E), which is located in California, much further north.
How do we know which sites are closer to the equator?
By checking the latitude coordinates:
CCAFS LC-40: 28.56°N
KSC LC-39A: 28.57°N
VAFB SLC-4E: 34.63°N
Comparison with Equator (Latitude 0°):
Florida is closer to the equator than California.
California is much farther north, which is not ideal for certain launches.
Why is latitude important?
 The Earth rotates from west to east, which gives an extra boost to rockets launched from latitudes closer to the Equator.
Less fuel = More efficiency and lower cost.
Geostationary (GEO) satellite launches are often conducted from lower latitudes to take advantage of this.

*   Are all launch sites in very close proximity to the coast?
  
Yes, all launch sites are in close proximity to the coast.
Main reason: This reduces the risk of falling debris over populated areas and allows for safer trajectories over the ocean.
How do we know the sites are in close proximity to the coast?
Map observation :
On the interactive map generated with Folium, we can see that the launch sites are very close to the ocean.
Actual location of the sites:
CCAFS LC-40 and KSC LC-39A → In Florida, at the edge of the Atlantic Ocean.
VAFB SLC-4E → In California, on the coast of the Pacific Ocean.
Why is it important to launch near the sea?
If a rocket fails after liftoff, the debris will fall into the sea rather than into populated areas.
Launching over the ocean allows for safer trajectories and reduces the risk of accidents.
Coastal bases facilitate launches in different directions without flying over cities or populated areas.

Also please try to explain your findings.
General Conclusion.
SpaceX chooses launch sites close to the sea to minimize risks in case of failures.
As far as possible, it selects sites at low latitudes, as the Earth's rotation provides extra momentum to rockets launched from areas close to the equator.
For these reasons, Florida is a strategic location for space launches.


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


In [14]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [15]:
# Task 2: Mark the success/failed launches for each site on the map
# Import the MarkerCluster plugin
from folium.plugins import MarkerCluster

# Create a Marker Cluster layer
marker_cluster = MarkerCluster().add_to(site_map)

# Loop through each row in the dataframe and add markers
for index, row in spacex_df.iterrows():
    # Extract information
    lat = row['Lat']
    lon = row['Long']
    launch_site = row['Launch Site']
    outcome = row['class']  # 1 = Success, 0 = Failure
    
    # Set marker color based on success or failure
    marker_color = "green" if outcome == 1 else "red"
    
    # Create marker with popup
    folium.Marker(
        location=[lat, lon],
        popup=f"Launch Site: {launch_site}\nOutcome: {'Success' if outcome == 1 else 'Failure'}",
        icon=folium.Icon(color=marker_color, icon="ok-sign" if outcome == 1 else "remove-sign")
    ).add_to(marker_cluster)

# Display the updated map
site_map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [16]:
# Import MarkerCluster
from folium.plugins import MarkerCluster

# Create a MarkerCluster object
marker_cluster = MarkerCluster()

# Add marker_cluster to the site_map
site_map.add_child(marker_cluster)

*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [17]:
# Define a function to assign colors based on class value
def assign_marker_color(launch_class):
    return 'green' if launch_class == 1 else 'red'

# Apply the function to create a new column 'marker_color'
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)

# Display the first few rows to confirm the new column
spacex_df.head()

,Launch Site,Lat,Long,class,marker_color
0,CCAFS LC-40,28.562302,-80.577356,0,red
1,CCAFS LC-40,28.562302,-80.577356,0,red
2,CCAFS LC-40,28.562302,-80.577356,0,red
3,CCAFS LC-40,28.562302,-80.577356,0,red
4,CCAFS LC-40,28.562302,-80.577356,0,red


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [18]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# For each row in spacex_df data frame
# Create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successful or failed
for index, record in spacex_df.iterrows():
    # Extract launch site coordinates
    launch_site = record['Launch Site']
    lat, lon = record['Lat'], record['Long']
    marker_color = record['marker_color']  # Use the assigned marker color
    
    # Create and add a marker with the specified color
    marker = folium.Marker(
        location=[lat, lon],
        popup=launch_site,
        icon=folium.Icon(color=marker_color, icon="ok-sign")
    )
    
    marker_cluster.add_child(marker)

# Display the updated map
site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [21]:
#Task3: Calculate the distances between a launch site to it oroximities
import math

# Define a function to calculate distance using Haversine formula
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of the Earth in km
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])  # Convert to radians

    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    return R * c  # Distance in km

# Dictionary of launch site coordinates
launch_sites_coords = {
    "KSC LC-39A": [28.573255, -80.648015],
    "CCAFS SLC-40": [28.563197, -80.576820],
    "VAFB SLC-4E": [34.632834, -120.610746]
}

# Example: Calculate distance between KSC LC-39A and CCAFS SLC-40
site1 = launch_sites_coords["KSC LC-39A"]
site2 = launch_sites_coords["CCAFS SLC-40"]

distance = haversine(site1[0], site1[1], site2[0], site2[1])

# Print the calculated distance
print(f"Distance between KSC LC-39A and CCAFS SLC-40: {distance:.2f} km")

Distance between KSC LC-39A and CCAFS SLC-40: 7.04 km


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [22]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [23]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [24]:
# Coordenadas del sitio de lanzamiento
launch_site_lat = 28.562302
launch_site_lon = -80.577356

# Coordenadas de la costa (obtenidas con MousePosition)
coastline_lat, coastline_lon = 28.56317, -80.56794

# Calcular la distancia al punto más cercano de la costa
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Mostrar el resultado
print(f"Distancia a la costa: {distance_coastline:.2f} km")

Distancia a la costa: 0.92 km


In [25]:
# Create and add a marker at the closest coastline point
distance_marker = folium.Marker(
    location=[coastline_lat, coastline_lon],
    icon=folium.DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:.2f} KM".format(distance_coastline),
    )
)

# Add the marker to the map
site_map.add_child(distance_marker)
site_map

*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [26]:
# Create a PolyLine using launch site and coastline coordinates
lines = folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]],
    weight=1
)

# Add the line to the map
site_map.add_child(lines)
site_map

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [27]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site
# Create and add a folium.Marker for the closest railway
railway_lat, railway_lon = 28.56261, -80.58708  # Railway coordinates
distance_railway = calculate_distance(launch_site_lat, launch_site_lon, railway_lat, railway_lon)

railway_marker = folium.Marker(
    location=[railway_lat, railway_lon],
    icon=folium.DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:.2f} KM".format(distance_railway),
    )
)

site_map.add_child(railway_marker)

In [28]:
# Create a folium.PolyLine to draw a line between the launch site and the railway
railway_line = folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [railway_lat, railway_lon]],
    weight=2,
    color='blue'
)

site_map.add_child(railway_line)

In [29]:
# Create and add a folium.Marker for the closest highway
highway_lat, highway_lon = 28.56189, -80.57051  # Highway coordinates
distance_highway = calculate_distance(launch_site_lat, launch_site_lon, highway_lat, highway_lon)

highway_marker = folium.Marker(
    location=[highway_lat, highway_lon],
    icon=folium.DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:.2f} KM".format(distance_highway),
    )
)

site_map.add_child(highway_marker)

# Create a folium.PolyLine to draw a line between the launch site and the highway
highway_line = folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [highway_lat, highway_lon]],
    weight=2,
    color='red'
)

site_map.add_child(highway_line)

After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
On the map, you can see that there is a rail line nearby. 
Proximity to railways
The launch sites are relatively close to railroad tracks. In the specific case of the analyzed site, there is a rail line that is approximately 0.95 km from the launch site. This infrastructure could be used to transport materials and equipment needed for launches.

*   Are launch sites in close proximity to highways?
You can see nearby roads connecting the launch site to other areas.
Proximity to roads 
Launch sites are connected to nearby roads, with a distance of approximately 0.95 km to the nearest road. This proximity facilitates the logistics of transporting personnel and supplies.


*   Are launch sites in close proximity to coastline?
Yes, measurements indicate that they are less than 1 km from the coast.
Proximity to the coast 
All launch sites are close to the coast. In the analysis performed, it is 0.92 km from the coastline. The proximity to the ocean is crucial to reduce risks in case of launch failures, allowing the debris to fall into the sea instead of populated areas.


*   Do launch sites keep certain distance away from cities?
Yes, the launch sites are in more isolated areas, far from major cities. Distance from cities 
Launch sites are usually located away from urban areas. This distance is a safety measure to minimize the impact of any incident during liftoff. It also reduces noise pollution and other negative environmental effects on the population.

Also please try to explain your findings.
conclusion
Launch sites are strategically located near the coast to minimize risks, and relatively close to key infrastructure such as roads and railroads to facilitate logistics. However, they maintain a safe distance from cities to avoid negative impacts on urban populations. These siting decisions play a critical role in the efficiency and safety of space launches.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
